# Model Compression using Quantization


In [ ]:
!pip install datasets transformers torch scikit-learn

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import BitsAndBytesConfig

In [2]:
data = load_dataset("Arnav0805/phishing-site-classification")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

In [3]:
device = torch.device('cuda')

In [4]:
model_id = "Arnav0805/bert-phishing-classifier_student"
tokenizer = AutoTokenizer.from_pretrained("Arnav0805/bert-phishing-classifier_teacher")
model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

tokenized_data = data.map(preprocess_function, batched=True)
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

### Evaluation

In [6]:
def evaluate_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    # Disable gradient calculations
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass to get logits
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get predictions
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    return accuracy, precision, recall, f1

### Evaluate model (pre-quantization)

In [7]:
validation_dataloader = DataLoader(tokenized_data['validation'], batch_size=128)

In [8]:
base_accuracy, base_precision, base_recall, base_f1 = evaluate_model(model, validation_dataloader, device)
print("Pre-quantization Performance")
print(f"Accuracy: {base_accuracy:.4f}, Precision: {base_precision:.4f}, Recall: {base_recall:.4f}, F1 Score: {base_f1:.4f}")

Pre-quantization Performance
Accuracy: 0.9244, Precision: 0.9484, Recall: 0.8978, F1 Score: 0.9224


### Quantize model

In [10]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model_nf4 = AutoModelForSequenceClassification.from_pretrained(model_id, device_map=device, quantization_config=nf4_config)

### Evaluate Model (post-quantization)

In [11]:
quantized_accuracy, quantized_precision, quantized_recall, quantized_f1 = evaluate_model(model_nf4, validation_dataloader, device)

print("Post-quantization Performance")
print(f"Accuracy: {quantized_accuracy:.4f}, Precision: {quantized_precision:.4f}, Recall: {quantized_recall:.4f}, F1 Score: {quantized_f1:.4f}")

Post-quantization Performance
Accuracy: 0.9267, Precision: 0.9444, Recall: 0.9067, F1 Score: 0.9252


In [14]:
from huggingface_hub import notebook_login

notebook_login()

In [15]:
model_nf4.push_to_hub("Arnav0805/bert-phishing-classifier_student_4bit")

model.safetensors:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Arnav0805/bert-phishing-classifier_student_4bit/commit/256654fbe9ee3b5a4152faca74920f48254d0824', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='256654fbe9ee3b5a4152faca74920f48254d0824', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Arnav0805/bert-phishing-classifier_student_4bit', endpoint='https://huggingface.co', repo_type='model', repo_id='Arnav0805/bert-phishing-classifier_student_4bit'), pr_revision=None, pr_num=None)